In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from Data.datasets import save_obj, load_obj
from scipy.optimize import minimize
import itertools
import sympy as sym

# load the homogeneous concentration space
R_conc = load_obj('8R homogeneous concentration statespace (Pb2, morph, H2O and FAH constrained)')
R_conc_stand = load_obj('8R homogeneous concentration statespace_standardized (Pb2, morph, H2O and FAH constrained)')
R_vol = load_obj('8R homogeneous volume statespace (Pb2, morph, H2O and FAH constrained)')

### Find initial sampling points

#### Initial sampling using kenard stone algorithm

In [ ]:
from Models import sampling

k = 48
## select initial training data using Kennard-Stone algorithm
sample_idx, _ = sampling.kennardstonealgorithm(R_conc_stand, k)

# use the sample_idx to select initial sampling from R_conc_exp and R_vol_exp
R_conc_stand_init = R_conc_stand.iloc[sample_idx]
R_conc_init = R_conc.loc[R_conc_stand_init.index]

save_obj(R_conc_stand_init, '8 reagent concentration_initial sampling_standardized')
save_obj(R_conc_init, '8 reagent concentration_initial sampling')

### Choose initial volume set using the index of R_conc_init

In [ ]:
R_vol_init = R_vol.loc[R_conc_init.index]
R_vol_init_a = R_vol_init.iloc[:24]
R_vol_init_b = R_vol_init.iloc[24:]
save_obj(R_vol_init, '8 reagent volume_initial sampling')

In [ ]:
R_vol_init = load_obj('8 reagent volume_initial sampling')
R_vol_init_a = R_vol_init.iloc[:24]
R_vol_init_b = R_vol_init.iloc[24:]
from Models.expgen import robot_file_gen_R8
robot_file_gen_R8(data = R_vol_init_a, filename = '8R_init_ks sampling_robotinput_part_a')
robot_file_gen_R8(data = R_vol_init_b, filename = '8R_init_ks sampling_robotinput_part_b')

In [ ]:
# Generate new robot input files
df_rbtinput=pd.read_excel('Models/RobotInput_template_8R.xls')

pipette_low = "Tip_50ul_Water_DispenseJet_Empty"
pipette_med = "StandardVolume_Water_DispenseJet_Empty"
pipette_high = "HighVolume_Water_DispenseJet_Empty"

df_rbtinput['Reagent1 (ul)'] = np.array(R_vol_init_b['R1']) # pure DMF
df_rbtinput['Reagent2 (ul)'] = np.array(R_vol_init_b['R2']) # pure DMSO
df_rbtinput['Reagent3 (ul)'] = np.array(R_vol_init_b['R3']) # pure GBL
df_rbtinput['Reagent4 (ul)'] = np.array(R_vol_init_b['R4']) # Pb/morph DMF
df_rbtinput['Reagent5 (ul)'] = np.array(R_vol_init_b['R5']) # morph DMF
df_rbtinput['Reagent7 (ul)'] = np.array(R_vol_init_b['R6']) # formic acid
df_rbtinput['Reagent8 (ul)'] = np.array(R_vol_init_b['R7']) # water


def liquid_class (vol):
    if max(vol) <= 50:
        output = pipette_low
    elif (max(vol) > 50) & (max(vol) <= 300):
        output = pipette_med
    elif (max(vol) > 300) & (max(vol) <= 1000):
        output = pipette_high
    else:
        raise ValueError("ValueError: volume given is out of range")
    return output

df_rbtinput['Liquid Class'][0] = liquid_class(df_rbtinput['Reagent1 (ul)'])
df_rbtinput['Liquid Class'][1] = liquid_class(df_rbtinput['Reagent2 (ul)'])
df_rbtinput['Liquid Class'][2] = liquid_class(df_rbtinput['Reagent3 (ul)'])
df_rbtinput['Liquid Class'][3] = liquid_class(df_rbtinput['Reagent4 (ul)'])
df_rbtinput['Liquid Class'][4] = liquid_class(df_rbtinput['Reagent5 (ul)'])
df_rbtinput['Liquid Class'][5] = liquid_class(df_rbtinput['Reagent6 (ul)'])
df_rbtinput['Liquid Class'][6] = liquid_class(df_rbtinput['Reagent7 (ul)'])
df_rbtinput['Liquid Class'][7] = liquid_class(df_rbtinput['Reagent8 (ul)'])
df_rbtinput['Liquid Class'][8] = liquid_class(df_rbtinput['Reagent9 (ul)'])

df_rbtinput.to_csv('8R_init_ks sampling_robotinput_part_b.csv')

In [ ]:
R_conc_stand_init = load_obj('8 reagent concentration_initial sampling_standardized')
R_conc_init = load_obj('8 reagent concentration_initial sampling')

In [ ]:
R_conc_stand_init.to_csv('8R_init_ks sampling_conc_stand.csv')
R_conc_init.to_csv('8R_init_ks sampling_conc.csv')